In [1]:
from torchvision import datasets
import torch
import torchvision.transforms as tf
import pdb
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
from MyNeuralNetwork import NeuralNetwork

In [3]:
def GetDatasets(download=True):
    dataset_train = datasets.FashionMNIST(
        root = r'data/train',
        download = download,
        train = True,
        transform = tf.ToTensor() 
    )
    
    dataset_test = datasets.FashionMNIST(
        root = r'data/test',
        download = download,
        train = False,
        transform = tf.ToTensor() 
    )
    return dataset_train, dataset_test

In [4]:
dataset_train, dataset_test = GetDatasets(False);

In [5]:
batch_size = 500
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [6]:
device = 'cpu'
network = NeuralNetwork().to(device)

In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=0.0001)

In [8]:
num_epochs = 10

In [9]:
for lr in [0.5, 0.05, 0.005, 0.0005]:
    optimizer = torch.optim.SGD(network.parameters(), lr=lr)
    
    log_dir = './logs/{}'.format(lr)
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(num_epochs):
        
        epoch_loss = 0
        epoch_accuracy = 0
        
        for idx, batch in enumerate(dataloader_train):
            # zero the gradient
            optimizer.zero_grad()

            x, y = batch[0], batch[1]

            # flatten it first
            x = torch.flatten(x, start_dim = 1)

            # forward pass
            predictions = network(x)

            loss = criterion(predictions, y)

            # backpropogate the loss
            loss.backward()

            # step the optimizer in the direction of the gradient
            optimizer.step()
            epoch_loss += loss.item()
        
        writer.add_scalar('Epoch loss', epoch_loss / idx, epoch)
        writer.flush()
        
        # debug following code
        for idx, batch in enumerate(dataloader_test):
            
            network.eval()
            
            x, y = batch[0].to(device), batch[1].to(device)
            x = torch.flatten(x, start_dim = 1)
            predcitions = network(x)
            
            pred_label = torch.argmax(predictions, dim=1)
            n_correct = (y==pred_label).float()
            accuracy = n_correct.sum() / n_correct.numel()
            epoch_accuracy += 100 * accuracy
            
        writer.add_scalar('Epoch accuracy', epoch_accuracy / idx, epoch)
        writer.flush()
        
        print('Learning rate: {}, Epoch loss: {:f}, accuracy: {:f}'.format(lr, epoch_loss / idx, epoch_accuracy))
        writer.close()


Learning rate: 0.5, Epoch loss: 2.064820403812312
Learning rate: 0.5, Epoch loss: 1.7848527701962895
Learning rate: 0.5, Epoch loss: 1.7167900610370797
Learning rate: 0.5, Epoch loss: 1.6957913496915031
Learning rate: 0.5, Epoch loss: 1.6859249978506265
Learning rate: 0.5, Epoch loss: 1.6811046199638302
Learning rate: 0.5, Epoch loss: 1.6740297369596338
Learning rate: 0.5, Epoch loss: 1.67105129085669
Learning rate: 0.5, Epoch loss: 1.6673994074348641
Learning rate: 0.5, Epoch loss: 1.6652306959408671
Learning rate: 0.05, Epoch loss: 1.6579954083226307
Learning rate: 0.05, Epoch loss: 1.6573620163092093
Learning rate: 0.05, Epoch loss: 1.6569696204001163
Learning rate: 0.05, Epoch loss: 1.6567368928123922
Learning rate: 0.05, Epoch loss: 1.6564426712629174
Learning rate: 0.05, Epoch loss: 1.6561844338889884
Learning rate: 0.05, Epoch loss: 1.6558740990502494
Learning rate: 0.05, Epoch loss: 1.6556233778721143
Learning rate: 0.05, Epoch loss: 1.6553639494070487
Learning rate: 0.05, Epoc